## 요약

- 도서명 전처리 다시 해야 함
    - 한자어 정리
        - 독음이 병기 된 경우까지 포함하여 해결 완료
        - hanja에 포함되지 않은 한자 해결 필요
    - 숫자 사이 , 정리 : e.g. 1,000 -> 1000
    - 기타 전처리
        - 로마 숫자를 아랍 숫자로 변환
        - 특수문자 detect 및 가까운 특수 문자로 변환
            - '&#'
        - 토큰 사전에 추가 필요 : 'C++', 'ㆍ'
    - 괄호 정리 코드 오류 수정

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# 그래프의 폰트 출력을 선명하게 (svg, retina 등이 있음)
matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("default")
# 이미지가 레이아웃 안으로 들어오도록 함
# https://matplotlib.org/stable/users/explain/axes/constrainedlayout_guide.html
mpl.rcParams.update({"figure.constrained_layout.use": True})

#font, line, marker 등의 배율 설정: paper, notebook, talk, poster
sns.set_context("paper") 
#배색 설정: tab10, Set2, Accent, husl
sns.set_palette("Set2") 
#눈금, 배경, 격자 설정: ticks, white, whitegrid, dark, darkgrid
# withegrid: 눈금을 그리고, 각 축의 눈금을 제거
sns.set_style("whitegrid") 

In [3]:
# 로컬에서 

plt.rc("font", family = "D2Coding")
plt.rcParams["axes.unicode_minus"] = False

In [4]:
import os, natsort, re
from tqdm import tqdm
import time, random

In [5]:
PRJCT_PATH = '/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_usedbook/'
save_dir = 'processed/'
file_name = f'bookinfo_ver0.csv'

In [6]:
file_path = os.path.join(PRJCT_PATH,save_dir,file_name)
bookinfo = pd.read_csv(file_path)
bookinfo

,Rank,BName,BName_sub,ItemId,Author,Author_mul,Publshr,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Sorce
0,322.0,민병철 생활영어 1,NaN,54947,민병철,False,기타 출판사,1979-10-01,5500,4950,177.0,외국어,2001년1월3주
1,767.0,행복에의 초대,NaN,168173,IVP(한국기독학생회출판부) 편집부,False,기타 출판사,1979-12-01,2800,2660,1960.0,종교/역학,2005년2월1주
2,449.0,훈련으로 되는 제자,NaN,458277,월터 A. 헨릭슨,False,기타 출판사,1980-12-01,4500,4050,552.0,종교/역학,2005년2월1주
3,490.0,전태일 평전,NaN,128759,조영래,False,돌베개,1983-06-01,6000,6000,409.0,사회과학,2001년1월3주
4,300.0,제자의 삶,NaN,131488,프랜시스 M.코스그로브,False,기타 출판사,1984-07-01,12000,12000,1110.0,종교/역학,2005년2월1주
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9323,68.0,장송의 프리렌 12,NaN,342597187,야마다 카네히토,True,학산문화사(만화),2024-07-25,6000,5400,10530.0,만화,2024년7월2주
9324,138.0,약사의 혼잣말 13,NaN,342597589,네코쿠라게,True,학산문화사(만화),2024-07-25,6000,5400,5870.0,만화,2024년7월2주
9325,129.0,타몬 군 지금 어느 쪽?! 7,NaN,342597714,시와스 유키,True,학산문화사(만화),2024-07-25,6000,5400,6370.0,만화,2024년7월2주
9326,209.0,천재 의사 시건방 1,NaN,342388828,강효미,True,기타 출판사,2024-07-31,13000,11700,8190.0,어린이,2024년7월2주


- 책제목, 장르에 들어있는 token들 파악

In [7]:
from konlpy.tag import Mecab
mecab = Mecab()
tokenizer_basic = lambda x : mecab.morphs(x)

# 출처: https://sikaleo.tistory.com/62 [SIKALEO:티스토리]
def tokenizer_kor(raw, pos=["NNG","NNP","VV","VA"]):
    from konlpy.tag import Mecab
    m = Mecab()
    # 길이가 1 이하인 토근은 제외, 위에서 지정한 (mecab 사전에 따른) 토큰들만 특징으로 삼기, 불용어 100개 제외  
    return [word for word, tag in m.pos(raw) if tag in pos]

In [8]:
book_name, category = bookinfo.BName, bookinfo.Category
book_name = book_name.apply(tokenizer_basic)
category = category.apply(tokenizer_basic)

display(book_name)
display(category)

0                    [민병철, 생활, 영어, 1]
1                      [행복, 에, 의, 초대]
2                  [훈련, 으로, 되, 는, 제자]
3                           [전태일, 평전]
4                          [제자, 의, 삶]
                    ...              
9323               [장송, 의, 프리, 렌, 12]
9324                 [약사, 의, 혼잣말, 13]
9325    [타, 몬, 군, 지금, 어느, 쪽, ?, !, 7]
9326             [천재, 의, 사, 시, 건방, 1]
9327         [전지, 적, 루이, &, 후, 이, 시점]
Name: BName, Length: 9328, dtype: object

0             [외국어]
1       [종교, /, 역학]
2       [종교, /, 역학]
3          [사회, 과학]
4       [종교, /, 역학]
           ...     
9323           [만화]
9324           [만화]
9325           [만화]
9326          [어린이]
9327          [에세이]
Name: Category, Length: 9328, dtype: object

In [9]:
import itertools
tokens = np.array(list(itertools.chain(*book_name.values,*category.values)))
len(tokens)

78746

In [10]:
corpus = np.unique(tokens,return_counts=True)
display(corpus)
display(len(corpus[0]))

(array(['!', '$', '%', ..., '老子', '漏泄', '／'], dtype='<U14'),
 array([190,   1,  14, ...,   1,   1,   2]))

8384

In [11]:
df_corpus = pd.DataFrame(corpus).T
df_corpus = df_corpus.rename(columns={0:'token',1:'counts'})
df_corpus

,token,counts
0,!,190
1,$,1
2,%,14
3,&,44
4,&#,1
...,...,...
8379,힙합,2
8380,洛陽,1
8381,老子,1
8382,漏泄,1


- 특수문자, 숫자, 한자 분리 - 이진탐색을 수동으로 진행함

In [12]:
df_corpus.iloc[:20]

,token,counts
0,!,190
1,$,1
2,%,14
3,&,44
4,&#,1
5,',22
6,(,6
7,),5
8,*,2
9,+,179


In [13]:
df_corpus.iloc[20:40]

,token,counts
20,07,1
21,08,1
22,1,757
23,10,117
24,100,69
25,1000,64
26,1001,1
27,1004,1
28,1006,1
29,101,8


In [14]:
df_corpus.iloc[40:235]

,token,counts
40,118,2
41,119,4
42,12,47
43,120,4
44,1200,11
...,...,...
230,97,2
231,98,1
232,99,6
233,999,2


In [15]:
df_corpus.iloc[234:241]

,token,counts
234,:,287
235,;,2
236,<,1
237,=,1
238,>,1
239,?,110
240,@,4


In [16]:
df_corpus.iloc[241:884]

,token,counts
241,A,7
242,AA,1
243,ABCD,1
244,ACL,27
245,ADHD,2
...,...,...
879,workbook,1
880,x,3
881,xFF,1
882,you,1


In [17]:
df_corpus.iloc[883:900]

,token,counts
883,|,1
884,~,49
885,~!,1
886,·,6
887,×,1
888,…,4
889,Ⅱ,1
890,Ⅲ,1
891,Ⅴ,1
892,Ⅵ,1


In [18]:
df_corpus.iloc[899:931]

,token,counts
899,上,2
900,代,1
901,伽藍,1
902,傳,1
903,分,1
904,反,1
905,商道,6
906,國王,2
907,大,1
908,天氣,1


In [19]:
df_corpus.iloc[931:8381]

,token,counts
931,가가,1
932,가가미,1
933,가게,1
934,가격,2
935,가계,1
...,...,...
8376,힘든,1
8377,힘들,5
8378,힙,1
8379,힙합,2


In [20]:
df_corpus.iloc[8380:]

,token,counts
8380,洛陽,1
8381,老子,1
8382,漏泄,1
8383,／,2


- 특이 케이스 확인

In [21]:
targets = [
'Ⅱ',
'Ⅲ',
'Ⅴ',
'Ⅵ',
'ⓔ',
'★',
'☆',
'「',
'」',
'ㆍ동형']
detect_target = lambda x : len(set(x).intersection(set(targets))) > 0
cond = book_name.apply(detect_target)
book_name[cond]

667                           [Tangerine, TOEIC, Part, Ⅴ]
724                           [tangerine, TOEIC, Part, Ⅵ]
1547                          [tangerine, TOEIC, Part, Ⅱ]
1548                          [tangerine, TOEIC, Part, Ⅲ]
3609                             [운수, 좋, 은, 날, 「, 현진건, 」]
3882                                              [역사, ⓔ]
5707                                   [몬스, ☆, 패닉, NG, 2]
5809                                   [플레잉, ☆, 보이, 부, 1]
5811                                        [스타, ☆, 플라이트]
5939    [큰, 별, 쌤, 최태성, 의, 별, ★, 별한, 국사, 한, 국사, 능력, 검정,...
5947    [큰, 별, 쌤, 최태성, 의, 별, ★, 별한, 국사, 한, 국사, 능력, 검정,...
5960    [큰, 별, 쌤, 최태성, 의, 별, ★, 별한, 국사, 한, 국사, 능력, 검정,...
5998                                   [플레잉, ☆, 보이, 부, 2]
6816                                   [플레잉, ☆, 보이, 부, 3]
7149    [2021, 큰, 별, 쌤, 최태성, 의, 별, ★, 별한, 국사, 기출, 500,...
7207    [2021, 큰, 별, 쌤, 최태성, 의, 별, ★, 별한, 국사, 한, 국사, 능...
7208    [2021, 큰, 별, 쌤, 최태성, 의, 별, ★, 별한, 국사, 한, 국사, 능...
7277    [2021,

In [22]:
bookinfo.loc[[3609,7750,8008,8716]]

,Rank,BName,BName_sub,ItemId,Author,Author_mul,Publshr,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Sorce
3609,685.0,운수 좋은 날 「현진건」,NaN,15225976,현진건,True,기타 출판사,2012-02-13,7000,6300,6742.0,소설/시/희곡,2024년7월2주
7750,825.0,나의 「소세키」와 「류노스케」,NaN,272574967,우치다 햣켄,True,기타 출판사,2021-06-08,16500,14850,107.0,인문학,2021년6월1주
8008,764.0,2022 민준호 독학 행정법 진도별ㆍ동형 모의고사,NaN,288295319,민준호,False,기타 출판사,2022-02-15,20000,18000,541.0,수험서/자격증,2022년5월2주
8716,31.0,2024 큰별쌤 최태성의 별★별한국사 기출 500제 한국사능력검정시험 심화,"(1, 2, 3급)",332133818,최태성,False,기타 출판사,2024-01-17,19500,17550,65893.0,수험서/자격증,2024년7월2주


- 7705 : 백슬래시 모양의 특수문자. 크롤링해도 똑같은 상황 발생할 것으로 추정
    - 별도 전처리 필요 or 사전 등록
- 'C++'의 경우 token을 나누지 않는 것이 더 낫지 않나?
- () 분리 문제: 괄호 안에 괄호가 있는 경우 제대로 정리가 되지 않은 상황 확인
- 숫자 : 숫자 단위간에 ,로 구분된 경우 어떻게 하면 좋을지
    - 일괄적으로 숫자 사이의 ','는 없애자

In [23]:
targets = ['&#','(',')','++','000','01','02','1026','1006']
detect_target = lambda x : len(set(x).intersection(set(targets))) > 0
cond = book_name.apply(detect_target)
book_name[cond]

391                              [New, 비주얼, C, ++, 6, 입문]
675                         [Beginning, Visual, C, ++, 6]
731                                           [아웃사이더, 02]
739                                             [구운몽, 01]
1095                                     [C, ++, 기초, 플러스]
1129                          [Visual, C, ++, 6, 완벽, 가이드]
1805                          [베트남, 10, ,, 000, 일, 의, 전쟁]
1826    [3, ,, 000, 만, 원, 으로, 시작, 하, 는, 부동산, 투자, 101, 가지]
1999                        [2, ,, 000, 원, 으로, 밥상, 차리, 기]
2097                       [5, ,, 000, 원, 으로, 손님상, 차리, 기]
2135                  [1, ,, 000, 원, 으로, 국, ,, 찌개, 만들, 기]
2998                                               [1026]
3226    [ETS, TOEIC, Listening, Prep, Book, (, 교재, +, ...
3227    [ETS, TOEIC, Reading, Prep, Book, (, 교재, +, 해설...
3608                           [일본어, 상용한자, 기초, 마스터, 1006]
5359    [시, 나공, TOEIC, New, 실전, 모의고사, 특별, 판, (, 신, 토익,...
6220                               [과, 호흡, (, 애니메이션, DVD]
7679    [인사, 반

In [24]:

pat = r'\d\,\d'
sentence = '10,000원 어치를 10.00% 세일하여 9,000원에 100,220,349,302개'

def erase_num_comma(text):
    idx_list = sorted([m.start()+1 for m in re.finditer(pat,text)])
    temp = list(text)
    for i in idx_list[::-1]:
        temp.pop(i)
    return ''.join(temp)

erase_num_comma(sentence)

'10000원 어치를 10.00% 세일하여 9000원에 100220349302개'

In [25]:
bookinfo.loc[[3226,3227,5359,6220,7679,7705,8513]]

,Rank,BName,BName_sub,ItemId,Author,Author_mul,Publshr,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Sorce
3226,916.0,ETS TOEIC Listening Prep Book (교재 + 해설집 + 무료 동...,(ETS X-File 빈출표현 수록),11822711,ETS,False,와이비엠,2011-06-10,19800,17820,4903.0,외국어,2013년4월4주
3227,101.0,ETS TOEIC Reading Prep Book (교재 + 해설집 + 무료 동영상...,(ETS X-File 빈출어휘 수록),11822739,ETS,False,와이비엠,2011-06-10,19800,17820,4554.0,외국어,2011년12월5주
5359,802.0,시나공 TOEIC New 실전 모의고사 특별판 (신토익 반영 4회분 문제집 + 자세...,"(新), (新)",83578927,백형식,False,길벗이지톡,2016-05-09,17800,16020,831.0,외국어,2016년7월3주
6220,486.0,과호흡 (애니메이션 DVD,(외전 포함)),168491261,뿡빵뀨,False,기타 출판사,2018-10-05,26800,24120,8396.0,만화,2019년1월3주
7679,181.0,인사반파자구계통 박스 세트 - 전3권 (스티커 + 엽서 세트 + 마스킹테이프 + L...,(4종),271933844,묵향동후,False,기타 출판사,2021-05-31,69800,62820,2289.0,소설/시/희곡,2021년6월1주
7705,730.0,세 명의 삶 &#xFF3C; Q. E. D.,NaN,269949898,거트루드 스타인,True,기타 출판사,2021-06-01,16000,14400,547.0,소설/시/희곡,2021년6월1주
8513,759.0,악당의 아빠를 꼬셔라 3 (캐릭터 책갈피 1종 + 투명 포토카드 2종 + DIY 편...,(패브릭 포스터),291737282,비아,True,기타 출판사,2022-05-27,35000,31500,360.0,만화,2022년5월2주


In [26]:
cond = bookinfo.BName.apply(lambda x : 'ㆍ' in x)
bookinfo[cond]

,Rank,BName,BName_sub,ItemId,Author,Author_mul,Publshr,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Sorce
8008,764.0,2022 민준호 독학 행정법 진도별ㆍ동형 모의고사,NaN,288295319,민준호,False,기타 출판사,2022-02-15,20000,18000,541.0,수험서/자격증,2022년5월2주


In [27]:
patterns=dict()
patterns['['] = r'\[+[^\[\]]+\]+'
patterns['{'] = r'\{+[^\{\}]+\}+'
patterns['('] = r'\(+[^\(\)]+\)+'

def re_iter_to_rslt(re_iter):
    return {
        m.start() : m.group()
        for m in re_iter
    }
    
def find_patterns(patterns,sentence):
    return {
        key : re_iter_to_rslt(re.finditer(pat,sentence.strip()))
        for key,pat in patterns.items()
    }

In [28]:
sentence = '안녕 (하이(하세요))'
find_patterns(patterns,sentence)

{'[': {}, '{': {}, '(': {6: '(하세요))'}}

In [29]:
sentence2= '악당의 아빠를 꼬셔라 3 (캐릭터 책갈피 1종 + 투명 포토카드 2종 + DIY 편지지 책자 + 컵홀더 2종 + 테이블 매트(패브릭 포스터) + 종이 코스터 2종 + 주접 대사 스티커 3종 + 박스 포함 한정판)'
patterns2=dict()
patterns2['['] = r'\[.+\]'
patterns2['{'] = r'\{.+\}'
patterns2['('] = r'\(.+\)'
display(find_patterns(patterns2,sentence))
display(find_patterns(patterns2,sentence2))

{'[': {}, '{': {}, '(': {3: '(하이(하세요))'}}

{'[': {},
 '{': {},
 '(': {14: '(캐릭터 책갈피 1종 + 투명 포토카드 2종 + DIY 편지지 책자 + 컵홀더 2종 + 테이블 매트(패브릭 포스터) + 종이 코스터 2종 + 주접 대사 스티커 3종 + 박스 포함 한정판)'}}

- 고민해봤는데 전처리 복잡하게 하지 말고, 괄호 겹치는 상태로 그냥 넣어버려도 될 듯
- pattern만 바꿔서 다시 파일 처리 필요

- 한자

In [30]:
display(df_corpus.iloc[899:930])
display(df_corpus.iloc[8380:8383])

,token,counts
899,上,2
900,代,1
901,伽藍,1
902,傳,1
903,分,1
904,反,1
905,商道,6
906,國王,2
907,大,1
908,天氣,1


,token,counts
8380,洛陽,1
8381,老子,1
8382,漏泄,1


In [31]:

targets = list(df_corpus.loc[899:929,'token'].values)+list(df_corpus.loc[8380:8382,'token'].values)
detect_target = lambda x : len(set(x).intersection(set(targets))) > 0
cond = book_name.apply(detect_target)
book_name[cond]

440                           [21, 세기, 희망, 은, 農, 에, 있, 다]
767                                               [商道, 1]
768                                               [商道, 2]
769                                               [商道, 3]
770                                               [商道, 4]
771                                               [商道, 5]
865                                        [知, 의, 편집, 공학]
966                                               [老子, 翼]
1035                                                  [美]
1090                                          [洛陽, 伽藍, 記]
1211                                 [商道, -, 전, 5, 권, 세트]
1591                                       [예수, 는, 神話, 다]
1636                                             [新, 군주론]
1776                                    [디지털, 카메라, 新, 입문]
1953                                         [십, 시, 일, 反]
2385                        [辛, 워드, 프로세서, 1, 급, 필기, 기출문제]
2392                        [辛, 워드, 프로세서, 2, 급, 필기, 기출문제]
2514          

In [32]:
bookinfo[cond]

,Rank,BName,BName_sub,ItemId,Author,Author_mul,Publshr,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Sorce
440,930.0,21세기 희망은 農에 있다,NaN,221123,정경식,True,기타 출판사,2000-02-07,8000,7200,101.0,과학,2001년1월3주
767,23.0,商道 1,NaN,254107,최인호,False,기타 출판사,2000-11-01,7500,6750,2551.0,소설/시/희곡,2001년1월3주
768,43.0,商道 2,NaN,254111,최인호,False,기타 출판사,2000-11-01,7500,6750,2681.0,소설/시/희곡,2001년1월3주
769,38.0,商道 3,NaN,254113,최인호,False,기타 출판사,2000-11-01,7500,6750,2951.0,소설/시/희곡,2001년1월3주
770,33.0,商道 4,NaN,254114,최인호,False,기타 출판사,2000-11-01,7500,6750,3074.0,소설/시/희곡,2001년1월3주
771,37.0,商道 5,NaN,254117,최인호,False,기타 출판사,2000-11-01,7500,6750,3086.0,소설/시/희곡,2001년1월3주
865,453.0,知의 편집공학,NaN,257871,마쓰오카 세이고,True,넥서스,2000-11-30,12900,11610,256.0,사회과학,2001년1월3주
966,449.0,老子翼,NaN,259510,초횡약후,True,기타 출판사,2000-12-18,12800,11520,93.0,인문학,2001년1월3주
1035,488.0,美,NaN,261214,마르시아스 심 외,False,현대문학,2001-01-01,8000,7200,235.0,소설/시/희곡,2001년1월3주
1090,604.0,洛陽伽藍記,(낙양가람기),261789,양현지,True,기타 출판사,2001-01-10,12000,10800,237.0,역사,2001년1월3주


- 한자 처리 : 만약 한자와 똑같은 발음의 한글이 앞 혹은 뒤에 반복된 경우 제거, 이외의 경우는 번역
    - 괄호 안에 내용이 있는 경우도 포함
    - ```pip install git+https://github.com/suminb/hanja.git``` : 간체, 일본어 한자까지 지원한다고 함

In [33]:
import hanja

custom_hanja ={
    '洛' : '락'
}

def is_hanja_custom(chr):
    return hanja.is_hanja(chr) or chr in custom_hanja

def split_hanja_custom(text):
    """주어진 문장을 한자로 된 구역과 그 이외의 문자로 된 구역으로 분리"""
    # TODO: Can we make this a bit prettier?
    if len(text) == 0:
        yield text
    else:
        ch = text[0]
        bucket = [ch]
        prev_state = is_hanja_custom(ch)

        for ch in text[1:]:
            state = is_hanja_custom(ch)

            if prev_state != state:
                yield "".join(bucket)
                bucket = [ch]
            else:
                bucket.append(ch)

            prev_state = state

        yield "".join(bucket)


In [34]:
sentence = '낙양가람기    (洛陽伽藍記) '
print(list(split_hanja_custom(sentence)))

['낙양가람기    (', '洛陽伽藍記', ') ']


In [36]:
sentence = '洛陽伽藍記 (낙양 가람기) '
print(list(split_hanja_custom(sentence)))
print(list(hanja.split_hanja(sentence)))

['洛陽伽藍記', ' (낙양 가람기) ']
['洛', '陽伽藍記', ' (낙양 가람기) ']


In [37]:
sentence = '洛陽伽藍記 (낙양 가람기) '
list(hanja.split_hanja(sentence))

['洛', '陽伽藍記', ' (낙양 가람기) ']

In [38]:
hanja.translate('洛陽伽藍記','substitution')

'洛양가람기'

In [39]:
import hanja
import re

parens = [
    ('',''),
    ('(',')'),
    ('{','}'),
    ('[',']'),
    ('（','）'),
    ('［','］'),
    ('｛','｝'),
    ('<','>'),
    ('〈','〉'),
    ('《','》'),
    ('「','」'),
    ('『','』'),
    ('〔','〕'),
    ('【','】'),
]

erase_space = lambda x: re.sub(r'\s+','',x)

def check_adj(word,target,sign:bool):
    #sign : position of word < position of target
    if len(target) > len(word) : return False
    if sign : sliced = word[len(word)-len(target):]
    else : sliced = word[:len(target)]
    if sliced == target : 
        return True
    return False

def is_hanja_words(segment):
    check = list(map(hanja.is_hanja,list(segment)))
    return all(check)


In [59]:
## TODO
# 1. 일부가 사전에 없어서 등의 이유로 독음이 1~2글자 차이나게 있는 경우
#    cf. 낙양유람기(물이름 낙이 hanja에 없음), 항복, 필사즉생
# 2. 괄호가 여러개 겹쳐져 있는 케이스까지 커버

def translate_hanja(sentence):
    #masking
    temp = erase_space(sentence)
    segs = list(split_hanja_custom(temp))
    ishanja = list(map(is_hanja_words,segs))
    for idx,(chr,flag) in enumerate(zip(segs,ishanja)):
        if not flag : continue
        sound = hanja.translate(chr,'substitution')
        #한자 뒤에 '(독음)'의 형태로 있는지 확인
        if idx < len(segs)-1:
            check_list = list(map(lambda x : x[0]+sound+x[1],parens))
            for ele in check_list:
                flag2 = check_adj(segs[idx+1],ele,idx+1<idx)
                if flag2 : segs[idx+1] = ' '*len(ele) + segs[idx+1][len(ele):]
            segs[idx] = sound
        #독음 뒤에 '(한자)'의 형태로 있는지 확인
        if 1 <= idx: 
            for par in parens:
                if not check_adj(segs[idx-1],sound+par[0],idx-1<idx) : continue
                if len(par[1]) > 1 and len(segs) < idx+1 : continue
                if (len(par[1])==0) or (segs[idx+1][:len(par[1])] == par[1]) :
                    segs[idx] = ' '*len(chr)
                    if par[0] : segs[idx-1] = segs[idx-1][:-len(par[0])]+' '*len(par[0])
                    if par[1] : segs[idx+1] = ' '*len(par[1])+segs[idx+1][len(par[1]):]
    masked = ''.join(segs)
    #apply mask to original sentence
    idx = 0
    rslt = list(sentence)
    for i,chr in enumerate(list(sentence)):
        if re.match(r'\s',chr) : continue #have to chage condition for all type of spaces
        rslt[i] = masked[idx]
        idx += 1
    return re.sub(r'\s+',' ',''.join(rslt)).strip()

In [54]:
sentence = '2014 국민건강보험공단 직무종합수행능력평가 150分 모의시험'
translate_hanja(sentence)

'2014 국민건강보험공단 직무종합수행능력평가 150분 모의시험'

In [60]:
sentence = '漢陽伽藍記	(한양 가람기)를 보면'
translate_hanja(sentence)

'한양가람기 를 보면'

In [61]:
sentence = '한양가람기 (漢陽伽藍記)에 의하면'
translate_hanja(sentence)

'한양가람기 에 의하면'

In [62]:
sentence = '한양가람기漢陽伽藍記를 읽고서'
translate_hanja(sentence)

'한양가람기 를 읽고서'

In [63]:
sentence = '통通박사 조병호의 성경과 고대정치'
translate_hanja(sentence)

'통 박사 조병호의 성경과 고대정치'

- 한자가 있던 자리에 공백이 들어가게 됐는데, tokenizer할 생각 하면 큰 문제가 아닐 수도 있을 것 같음

- 사전에 추가해야할 단어 : 'ㆍ'
- 다른 단어로 변환 필요 :
    - '&#xFF3C;' (```&#xFF3C;```) : '\\'
        - 이후에도 전처리 과정에서 &#를 detect 먼저 해보는 것 필요
    - 로마자 숫자 : 아랍숫자

In [66]:
roman_number ={
   'ⅰ':'1',
   'ⅱ':'2',
   'ⅲ':'3',
   'ⅳ':'4',
   'ⅴ':'5',
   'ⅵ':'6',
   'ⅶ':'7',
   'ⅷ':'8',
   'ⅸ':'9',
   'ⅹ':'10',
   'Ⅰ':'1',
   'Ⅱ':'2',
   'Ⅲ':'3',
   'Ⅳ':'4',
   'Ⅴ':'5',
   'Ⅵ':'6',
   'Ⅶ':'7',
   'Ⅷ':'8',
   'Ⅸ':'9',
   'Ⅹ':'10',
}

special_chr = {
  '&#xFF3C;' : '\\',
}

In [70]:
def replace_by_dict(text,chr_dict):
    for key,val in chr_dict.items():
        text = text.replace(key,val)
    return text

In [71]:
bookinfo['BName'] = bookinfo['BName'].apply(lambda x : replace_by_dict(x,roman_number))

In [72]:
bookinfo.loc[[667,724]]

,Rank,BName,BName_sub,ItemId,Author,Author_mul,Publshr,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Sorce
667,582.0,Tangerine TOEIC Part 5,NaN,247462,최종민,False,기타 출판사,2000-09-06,9800,8820,1132.0,외국어,2002년8월3주
724,419.0,tangerine TOEIC Part 6,NaN,251427,최종민,False,기타 출판사,2000-10-11,9800,8820,1031.0,외국어,2002년8월3주


In [73]:
bookinfo['BName'] = bookinfo['BName'].apply(lambda x : replace_by_dict(x,special_chr))

In [74]:
bookinfo.loc[7705]

Rank                      730.0
BName         세 명의 삶 \ Q. E. D.
BName_sub                   NaN
ItemId                269949898
Author                 거트루드 스타인
Author_mul                 True
Publshr                  기타 출판사
Pdate                2021-06-01
RglPrice                  16000
SlsPrice                  14400
SalesPoint                547.0
Category                소설/시/희곡
Sorce                 2021년6월1주
Name: 7705, dtype: object